In [ ]:
from PIL import Image
import os, glob, cv2, random
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import applications, optimizers, losses, layers
from tensorflow.keras.layers import Input, concatenate, Dense, LeakyReLU, UpSampling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.optimizers import Adam
from IPython.display import clear_output

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
ramdom_seed = 4885
tf.random.set_random_seed(ramdom_seed)
np.random.seed(ramdom_seed)
random.seed(ramdom_seed)

In [ ]:
def augment_image_colorspace(image):
    # input shape [h, w * seq , c]
    random_gamma = random.uniform(0.8,1.2)
    random_brightness = random.uniform(0.5, 2.0)
    random_colors = [random.uniform(0.8,1.2), random.uniform(0.8,1.2), random.uniform(0.8,1.2)]
    # Randomly shift gamma.
    image_aug = image**random_gamma
    # Randomly shift brightness. 
    image_aug *= random_brightness
    # Randomly shift color. 
    white = np.ones([image.shape[0], image.shape[1]])
    color_image = np.stack([white * random_colors[i] for i in range(3)], axis=2)
    image_aug *= color_image
    # Saturate.
    image_aug = np.clip(image_aug, 0, 1)
    return image_aug

In [ ]:
def data_generator(data_path, batch, aug):
    dir_list = os.listdir(data_path)
    img_list = []
    for j in dir_list:
        img_path = os.path.join(data_path, j)
        tmp_list = sorted(glob.glob(img_path +'/*.png'))
        img_list += tmp_list
    idx = 0
    while 1:
        idx_list = list(range(0,len(img_list)))
        random.shuffle(idx_list)
        bat_img = []
        bat_lab = []
        if idx > len(idx_list) - batch:
            idx = idx_list[idx:]
            idx = 0
        else:
            tmp_list = idx_list[idx:idx+batch]
            idx = idx + batch
            
        for i in tmp_list:
            img_tmp = cv2.imread(img_list[i])
            
            if img_tmp is None:
                print(img_list[i])
            if aug == 1:    
                img = augment_image_colorspace(cv2.cvtColor(img_tmp[:,:256,:],cv2.COLOR_BGR2RGB)/255.)
            else:
                img = cv2.cvtColor(img_tmp[:,:256,:],cv2.COLOR_BGR2RGB)/255.
            lab = cv2.cvtColor(img_tmp[:,256:,:], cv2.COLOR_BGR2GRAY)/255.
            bat_img.append(img)
            bat_lab.append(lab) 
        yield np.array(bat_img), [np.expand_dims(np.array([cv2.resize(i, (256, 256)) for i in bat_lab]),axis = -1),
                                 np.expand_dims(np.array([cv2.resize(i, (128, 128)) for i in bat_lab]),axis = -1),
                                 np.expand_dims(np.array([cv2.resize(i, (64, 64)) for i in bat_lab]),axis = -1),
                                 np.expand_dims(np.array([cv2.resize(i, (32, 32)) for i in bat_lab]),axis = -1),]

In [ ]:
def _depthnet(inputs):
    conv1 = Conv2D(64, 7, padding = 'same', strides=(2, 2), activation = 'relu')(inputs)
    conv2 = Conv2D(128, 5,  padding = 'same', strides=(2, 2), activation = 'relu')(conv1)
    conv3 = Conv2D(256, 5,  padding = 'same', strides=(2, 2), activation = 'relu')(conv2)
    conv3_1 = Conv2D(256, 3, padding = 'same', strides=(1, 1), activation = 'relu')(conv3)
    conv4 = Conv2D(512, 3,  padding = 'same', strides=(2, 2), activation = 'relu')(conv3_1)
    conv4_1 = Conv2D(512, 3,  padding = 'same', strides=(1, 1), activation = 'relu')(conv4)
    conv5 = Conv2D(512, 3,  padding = 'same', strides=(2, 2), activation = 'relu')(conv4_1)
    conv5_1 = Conv2D(512, 3, padding = 'same', strides=(1, 1), activation = 'relu')(conv5)
    
    deconv5 = Conv2D(512, 3, padding = 'same', use_bias=False, strides=(1, 1), activation = 'relu')(UpSampling2D(size = (2,2), interpolation = 'bilinear')(conv5_1))
    deconv4 = Conv2D(256, 3, padding = 'same', use_bias=False, strides=(1, 1), activation = 'relu')(UpSampling2D(size = (2,2), interpolation = 'bilinear')(deconv5))
    merge4 = concatenate([deconv4,conv3], axis = 3)    
    aux3 = Conv2D(1, 3, padding = 'same', use_bias =False, strides=(1, 1), activation = 'sigmoid')(merge4)
    deconv3 = Conv2D(128, 3, padding = 'same', use_bias=False, strides=(1, 1), activation = 'relu')(UpSampling2D(size = (2,2), interpolation = 'bilinear')(merge4))
    merge3 = concatenate([deconv3,conv2], axis = 3)
    aux2 = Conv2D(1, 3, padding = 'same', use_bias =False, strides=(1, 1), activation = 'sigmoid')(merge3)
    deconv2 = Conv2D(64, 3, padding = 'same', use_bias=False, strides=(1, 1), activation = 'relu')(UpSampling2D(size = (2,2), interpolation = 'bilinear')(merge3))
    merge2 = concatenate([deconv2,conv1], axis = 3)
    aux1 = Conv2D(1, 3, padding = 'same', use_bias =False, strides=(1, 1), activation = 'sigmoid')(merge2)
    deconv1 = Conv2D(64, 3, padding = 'same', use_bias=False, strides=(1, 1), activation = 'relu')(UpSampling2D(size = (2,2), interpolation = 'bilinear')(merge2))
    output = Conv2D(1, 3, activation = 'sigmoid', padding = 'same')(deconv1)
    
    model = Model(inputs = inputs, outputs = [output, aux1, aux2, aux3])
    return model

def _egonet(inputs, frame):
    conv1 = Conv2D(16, 7, padding = 'same', strides=(2, 2), activation = 'relu')(inputs)
    conv2 = Conv2D(32, 5,  padding = 'same', strides=(2, 2), activation = 'relu')(conv1)
    conv3 = Conv2D(64, 5,  padding = 'same', strides=(2, 2), activation = 'relu')(conv2)
    conv4 = Conv2D(128, 3, padding = 'same', strides=(2, 2), activation = 'relu')(conv3)
    conv5 = Conv2D(256, 3,  padding = 'same', strides=(2, 2), activation = 'relu')(conv4)
    conv6 = Conv2D(256, 3,  padding = 'same', strides=(2, 2), activation = 'relu')(conv5)
    conv7 = Conv2D(256, 3,  padding = 'same', strides=(2, 2), activation = 'relu')(conv6)

    egomotion_pred = Conv2D(frame*6, 3, activation = 'sigmoid', padding = 'same')(conv7)
    
    model = Model(inputs = inputs, outputs = egomotion_pred)
    return model

In [ ]:
depthnet_input = Input(shape=(256,256,3))
egonet_input = Input(shape=(256,256,3))
depthnet = _depthnet(depthnet_input)
egonet = _egonet(egonet_input)
model.compile(optimizer = Adam(lr = 0.0002), loss = [SSIM_cs,SSIM_cs,SSIM_cs,SSIM_cs], loss_weights = [0.64, 0.16, 0.04, 0.01])
model.compile(optimizer = Adam(lr = 0.0002), loss = [SSIM_cs,SSIM_cs,SSIM_cs,SSIM_cs], loss_weights = [0.64, 0.16, 0.04, 0.01])

In [ ]:
model.summary()

In [ ]:
train_path = '/data1/depth/endo_sim2/train/'
batch = 32
train = data_generator(train_path, batch, 1)
val_path = '/data1/depth/endo_sim2/val/'
batch = 32
val = data_generator(val_path, batch, 0)

In [ ]:
dir_list = os.listdir(train_path)
train_img_list = []
for j in dir_list:
    img_path = os.path.join(train_path, j)
    tmp_list = sorted(glob.glob(img_path +'/*.png'))
    train_img_list += tmp_list
    
print(len(train_img_list))

In [ ]:
model_name = 'cv_dcv_aux'
scale = 10
steps = int(np.ceil(len(train_img_list)/batch))
epochs = 16

total_loss = []
val_total_loss = []

for epoch in range(epochs):
    for step in range(steps):
        train_img, train_lab = next(train)
        loss = model.train_on_batch(train_img, train_lab)
    total_loss.append(loss[0])
    val_img, val_lab = next(val)
    val_loss = model.evaluate(val_img, val_lab)
    
    val_total_loss.append(val_loss[0])
    plt.cla()
    plt.clf()
    plt.close()
    clear_output(wait=True)
    val_p = model.predict(val_img)
    
    
    plt.figure(figsize=(20, 20))
    plt.subplot(211)
    plt.plot(total_loss,'.-')
    plt.plot(val_total_loss,'.-')
    plt.legend(['Train', 'Validation'], loc='upper right')

    plt.subplot(234)
    plt.imshow(val_img[0])
    plt.title("input")
    plt.subplot(235)
    plt.imshow(1-np.reshape(val_p[0][0]*scale, (256,256)), cmap = 'viridis')
    plt.title("predict images(val)")
    plt.subplot(236)
    plt.imshow(1-np.reshape(val_lab[0][0]*scale, (256,256)), cmap = 'viridis')
    plt.title("label")
    plt.show()
    if epoch % 5 == 0:
        model_path = "checkpoints_cv_ssim/test_%s_%02d_%.5f.h5"%(model_name, epoch+1, loss[0])
        model.save_weights(model_path)

In [ ]:
model.load_weights('checkpoints_cv_ssim/test_cv_dcv_aux_11_0.00203.h5')

In [ ]:
scale = 2

test_img, test_lab= next(val)
test_p = model.predict(test_img)
plt.figure(figsize=(15, 15))
plt.subplot(141)
plt.imshow(test_img[0])
plt.title("input")
plt.subplot(142)
plt.imshow(1-np.reshape(test_p[0][0]*scale,(256,256)),vmin = 0, vmax = 1, cmap = 'viridis')
plt.title("val predict images(scaling)")
plt.subplot(143)
plt.imshow(1-np.reshape(test_lab[0][0]*scale,(256,256)),vmin = 0, vmax = 1, cmap = 'viridis')
plt.title("label(scaling)")
plt.subplot(144)
plt.imshow(1-np.reshape(test_lab[0][0], (256,256)),vmin = 0, vmax = 1, cmap = 'viridis')
plt.title("label")
plt.show()

In [ ]:
test_path = '/data1/depth/endo_sim2/test/'
batch = 32
test = data_generator(test_path, batch, 0)

In [ ]:
scale = 2
for i in range(10):
    test_img, test_lab= next(test)
    test_p = model.predict(test_img)
    plt.figure(figsize=(15, 15))
    plt.subplot(141)
    plt.imshow(test_img[0])
    plt.title("input")
    plt.subplot(142)
    plt.imshow(1-np.reshape(test_p[0][0]*scale,(256,256)),vmin = 0, vmax = 1, cmap = 'viridis')
    plt.title("test predict images(scaling)")
    plt.subplot(143)
    plt.imshow(1-np.reshape(test_lab[0][0]*scale,(256,256)),vmin = 0, vmax = 1, cmap = 'viridis')
    plt.title("label(scaling)")
    plt.subplot(144)
    plt.imshow(1-np.reshape(test_lab[0][0], (256,256)),vmin = 0, vmax = 1, cmap = 'viridis')
    plt.title("label")
    plt.show()

In [ ]:
from skimage.measure import compare_mse as mse
from skimage.measure import compare_nrmse as nrmse
from skimage.measure import compare_psnr as psnr
from skimage.measure import compare_ssim as ssim

In [ ]:
nrmse_ = []
mse_ = []
psnr_ = []
ssim_ = []

for i in range(100):
    test_img, test_lab= next(test)
    tmp_test = model.predict(test_img)
    tmp_test = tmp_test[0][0]
    tmp_test = tmp_test.astype('float64')
    nrmse_.append(nrmse(tmp_test, test_lab[0][0]))
    mse_.append(mse(tmp_test, test_lab[0][0]))
    psnr_.append(psnr(tmp_test, test_lab[0][0]))
    ssim_.append(ssim(tmp_test, test_lab[0][0], multichannel=True))

In [ ]:
print('rmse : ', np.array(nrmse_).mean())
print('mse : ', np.array(mse_).mean())
print('psnr : ', np.array(psnr_).mean())
print('ssim : ', np.array(ssim_).mean())

In [ ]:
path = '/data1/depth/endo/endo/endo_ori/train/'
dir_list = os.listdir(path)
dir_list.sort()

In [ ]:
total_list = []
for j in dir_list:
    frame_list = glob.glob(os.path.join(path, j, '*.png'))
    total_list += frame_list

In [ ]:
rand_list = random.sample(total_list, 10)
i = 1
for tmp in rand_list:
    img = np.expand_dims(cv2.resize(np.array(Image.open(tmp))/255., (256,256)), axis=0)
    test_p = model.predict(img)
    plt.figure(figsize=(15, 15))
    plt.subplot(121)
    plt.imshow(img[0])
    plt.title("input")
    plt.subplot(122)
    plt.imshow(1-np.reshape(test_p[0][0]*scale,(256,256)),vmin = 0, vmax = 1, cmap = 'viridis')
    plt.title("predict images(test)")
    plt.show()
    img_name = "sample1/img_%02d.png"%(i)
    predic_name = "sample1/predic_%02d.png"%(i)
    cv2.imwrite(img_name, cv2.cvtColor(cv2.normalize(img[0], None, 255, 0, cv2.NORM_MINMAX, cv2.CV_8UC1), cv2.COLOR_RGB2BGR))
    cv2.imwrite(predic_name, cv2.cvtColor(cv2.normalize(test_p[0][0], None, 255, 0, cv2.NORM_MINMAX, cv2.CV_8UC1), cv2.COLOR_GRAY2BGR))
    i += 1

In [ ]:
path = '/data1/GIANA/Detection/jerry/raw_image/'
dir_list = os.listdir(path)
dir_list.sort()

In [ ]:
frame_list = glob.glob(os.path.join(path, '*.png'))
rand_list = random.sample(frame_list, 10)
for tmp in rand_list:
    img = np.expand_dims(cv2.resize(np.array(Image.open(tmp))/255., (256,256)), axis=0)
    test_p = model.predict(img)
    plt.figure(figsize=(15, 15))
    plt.subplot(121)
    plt.imshow(img[0])
    plt.title("input")
    plt.subplot(122)
    plt.imshow(np.reshape(1- test_p[0][0]*scale,(256,256)),vmin = 0, vmax = 1, cmap = 'viridis')
    plt.title("predict images(test)")
    plt.show()

In [ ]:
def data_generator_test(data_path, batch, strat_frame, skip_frame, aug):
    dir_list = os.listdir(data_path)
    img_list = []
    for j in dir_list:
        img_path = os.path.join(data_path, j)
        tmp_list = sorted(glob.glob(img_path +'/*.png'))
        img_list += tmp_list
    img_list = img_list[strat_frame:]
    idx = 0
    while 1:
        idx_list = list(range(0,len(img_list),skip_frame))
        bat_img = []
        bat_lab = []
        if idx > len(idx_list) - batch:
            idx = idx_list[idx:]
            idx = 0
        else:
            tmp_list = idx_list[idx:idx+batch]
            idx = idx + batch
            
        for i in tmp_list:
            img_tmp = cv2.imread(img_list[i])
            
            if img_tmp is None:
                print(img_list[i])
            if aug == 1:    
                img = augment_image_colorspace(cv2.cvtColor(img_tmp[:,:256,:],cv2.COLOR_BGR2RGB)/255.)
            else:
                img = cv2.cvtColor(img_tmp[:,:256,:],cv2.COLOR_BGR2RGB)/255.
            lab = cv2.cvtColor(img_tmp[:,256:,:], cv2.COLOR_BGR2GRAY)/255.
            bat_img.append(img)
            bat_lab.append(lab) 
        yield np.array(bat_img), [np.expand_dims(np.array([cv2.resize(i, (256, 256)) for i in bat_lab]),axis = -1),
                                 np.expand_dims(np.array([cv2.resize(i, (128, 128)) for i in bat_lab]),axis = -1),
                                 np.expand_dims(np.array([cv2.resize(i, (64, 64)) for i in bat_lab]),axis = -1),
                                 np.expand_dims(np.array([cv2.resize(i, (32, 32)) for i in bat_lab]),axis = -1),]

In [ ]:
test_path = '/data1/depth/endo_sim2/test/'
batch = 1
strat_frame = 1000
skip_frame = 10
test = data_generator_test(test_path, batch, strat_frame, skip_frame, 0)

In [ ]:
scale = 2
for i in range(10):
    test_img, test_lab= next(test)
    test_p = model.predict(test_img)
    plt.figure(figsize=(15, 15))
    plt.subplot(151)
    plt.imshow(test_img[0])
    plt.title("input")
    plt.subplot(152)
    plt.imshow(1-np.reshape(test_p[0][0]*scale,(256,256)),vmin = 0, vmax = 1, cmap = 'viridis')
    plt.title("test predict images(scaling)")
    plt.subplot(153)
    plt.imshow(1-np.reshape(test_lab[0][0]*scale,(256,256)),vmin = 0, vmax = 1, cmap = 'viridis')
    plt.title("label(scaling)")
    plt.subplot(154)
    plt.imshow(1-np.reshape(test_lab[0][0], (256,256)),vmin = 0, vmax = 1, cmap = 'viridis')
    plt.title("label")
    plt.subplot(155)
    plt.imshow(np.reshape(test_p[0][0],(256,256)) - np.reshape(test_lab[0][0], (256,256))/np.reshape(test_lab[0][0], (256,256)),vmin = -1, vmax = 1, cmap = 'gray')
    plt.title("error map")
    plt.show()
    img_name = "sample/img_%02d.png"%(i)
    lab_name = "sample/lab_%02d.png"%(i)
    predic_name = "sample/predic_%02d.png"%(i)
    cv2.imwrite(img_name, cv2.cvtColor(cv2.normalize(test_img[0], None, 255, 0, cv2.NORM_MINMAX, cv2.CV_8UC1), cv2.COLOR_RGB2BGR))
    cv2.imwrite(lab_name, cv2.cvtColor(cv2.normalize(test_lab[0][0], None, 255, 0, cv2.NORM_MINMAX, cv2.CV_8UC1), cv2.COLOR_GRAY2BGR))
    cv2.imwrite(predic_name, cv2.cvtColor(cv2.normalize(test_p[0][0], None, 255, 0, cv2.NORM_MINMAX, cv2.CV_8UC1), cv2.COLOR_GRAY2BGR))